In [15]:
import requests
import pandas as pd
from config import census_key
import json

In [16]:
# Read in zip codes
zips = pd.read_csv("../Zip_Codes_Chicago.csv")
zips

,the_geom,OBJECTID,ZIP,SHAPE_AREA,SHAPE_LEN
0,MULTIPOLYGON (((-87.67762151065281 41.91775780...,33,60647,1.060523e+08,42720.044406
1,MULTIPOLYGON (((-87.72683253163021 41.92264626...,34,60639,1.274761e+08,48103.782721
2,MULTIPOLYGON (((-87.78500237831095 41.90914785...,35,60707,4.506904e+07,27288.609612
3,MULTIPOLYGON (((-87.6670686895295 41.888851884...,36,60622,7.085383e+07,42527.989679
4,MULTIPOLYGON (((-87.70655631674127 41.89555340...,37,60651,9.903962e+07,47970.140153
...,...,...,...,...,...
56,MULTIPOLYGON (((-87.69479284658739 41.83007507...,57,60623,1.552855e+08,53406.915617
57,MULTIPOLYGON (((-87.6830596746508 41.757859474...,58,60629,2.111148e+08,58701.325375
58,MULTIPOLYGON (((-87.62372919454222 41.72167067...,59,60620,2.116961e+08,58466.160298
59,MULTIPOLYGON (((-87.57691454500865 41.79510541...,60,60637,1.254243e+08,52377.854541


In [22]:
# Remove unneeded columns. This will create an error if rows have already been dropped
zips_trim = zips[['OBJECTID','ZIP']]

In [23]:
# Just checking in 
pd.set_option("display.max_rows", None, "display.max_columns", None)
print(zips_trim)

    OBJECTID    ZIP
0         33  60647
1         34  60639
2         35  60707
3         36  60622
4         37  60651
5         38  60611
6         39  60638
7         40  60652
8          1  60626
9          8  60615
10         9  60621
11         2  60645
12        13  60643
13         3  60660
14         4  60640
15        32  60614
16        41  60631
17        42  60646
18        43  60628
19         5  60625
20         6  60641
21         7  60657
22        10  60636
23        11  60649
24        12  60617
25        14  60633
26        15  60643
27        16  60612
28        48  60604
29        17  60624
30        47  60656
31        18  60644
32        49  60655
33        19  60603
34        20  60605
35        21  60653
36        22  60609
37        23  60666
38        24  60618
39        25  60616
40        26  60602
41        27  60601
42        28  60608
43        29  60607
44        30  60661
45        31  60606
46        44  60827
47        45  60630
48        46  60642


In [24]:
# reset print options
pd.reset_option('all')
zips_trim


: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



C:\Users\Ward33\anaconda3\lib\site-packages\pandas\_config\config.py:620: FutureWarning: 
: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.

  warnings.warn(d.msg, FutureWarning)


,OBJECTID,ZIP
0,33,60647
1,34,60639
2,35,60707
3,36,60622
4,37,60651
...,...,...
56,57,60623
57,58,60629
58,59,60620
59,60,60637


In [30]:
# API call loop for table B14005 2018
# some setup before the loop
#set base url for 2018
base_url = 'https://api.census.gov/data/2018/acs/acs5?'
group = "B15003"
zip_ = ""
selectB15003_full = []
# loop through each zip code
for index, row in zips_trim.iterrows():
    zip_ = row['ZIP']
    try:
        print(f"Trying {row['ZIP']}")
        response = requests.get(f"{base_url}get=group({group})&for=zip%20code%20tabulation%20area:{zip_}&key={census_key}").json()
    except:
        print(f"failed with this address: {base_url}get=group({group})&for=zip%20code%20tabulation%20area:{zip_}&key={census_key}")
    # Clean call response
    response_df = pd.DataFrame(response)
    response_df.columns = response_df.iloc[0]
    # Select needed columns
    selectB15003 = pd.DataFrame(response_df.iloc[1][['GEO_ID',
                    'B15003_001E','B15003_002E','B15003_012E','B15003_013E',
                     'B15003_014E','B15003_015E','B15003_016E','B15003_017E',
                    'B15003_018E','B15003_019E', 'B15003_020E','B15003_021E',
                    'B15003_022E','B15003_023E','B15003_024E','B15003_025E']])
    select2B15003 = selectB15003.transpose()
    # store selected response
    selectB15003_full.append(select2B15003.iloc[0])
# put all responses into a dataframe
selectB15003_full_df = pd.DataFrame(selectB15003_full)

selectB15003_full_df 
    

Trying 60647
Trying 60639
Trying 60707
Trying 60622
Trying 60651
Trying 60611
Trying 60638
Trying 60652
Trying 60626
Trying 60615
Trying 60621
Trying 60645
Trying 60643
Trying 60660
Trying 60640
Trying 60614
Trying 60631
Trying 60646
Trying 60628
Trying 60625
Trying 60641
Trying 60657
Trying 60636
Trying 60649
Trying 60617
Trying 60633
Trying 60643
Trying 60612
Trying 60604
Trying 60624
Trying 60656
Trying 60644
Trying 60655
Trying 60603
Trying 60605
Trying 60653
Trying 60609
Trying 60666
https://api.census.gov/data/2018/acs/acs5?get=group(B15003)&for=zip%20code%20tabulation%20area:60666&key=9c5aa4a34753830c81ff60db8991b08b03d03dd5
Trying 60618
Trying 60616
Trying 60602
Trying 60601
Trying 60608
Trying 60607
Trying 60661
Trying 60606
Trying 60827
Trying 60630
Trying 60642
Trying 60659
Trying 60707
Trying 60634
Trying 60613
Trying 60610
Trying 60654
Trying 60632
Trying 60623
Trying 60629
Trying 60620
Trying 60637
Trying 60619


,GEO_ID,B15003_001E,B15003_002E,B15003_012E,B15003_013E,B15003_014E,B15003_015E,B15003_016E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E
1,8600000US60647,62961,1020,1012,563,539,742,923,10657,1318,2066,5678,2786,20968,8502,2467,951
1,8600000US60639,56047,2660,1982,2047,1374,1599,2526,14920,2286,2394,6906,2996,4930,1631,355,108
1,8600000US60707,29881,522,645,400,628,419,907,7406,823,1531,5065,2493,5228,2054,481,158
1,8600000US60622,40825,627,340,213,360,204,251,3352,492,950,3434,1493,17957,7025,2305,962
1,8600000US60651,39140,935,1209,636,1259,1750,1378,11485,1255,2297,6233,2770,3721,1448,243,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,8600000US60623,50229,2134,1882,2407,1784,2296,1908,12209,1821,1801,6498,2405,3650,954,177,162
1,8600000US60629,67343,2328,2099,1586,1864,1999,1853,23545,3060,2740,9152,3475,4702,1289,79,113
1,8600000US60620,47028,328,799,652,1114,1735,1484,12691,1730,2964,11264,4067,4987,2217,290,162
1,8600000US60637,27943,292,314,400,901,1027,579,5213,804,1660,5494,1630,3923,2897,803,1735


In [36]:
# Create zip code column
selectB15003_full_df['ZIP'] = selectB15003_full_df['GEO_ID'].str[-5:]
selectB15003_full_df

,GEO_ID,B15003_001E,B15003_002E,B15003_012E,B15003_013E,B15003_014E,B15003_015E,B15003_016E,B15003_017E,B15003_018E,B15003_019E,B15003_020E,B15003_021E,B15003_022E,B15003_023E,B15003_024E,B15003_025E,ZIP
1,8600000US60647,62961,1020,1012,563,539,742,923,10657,1318,2066,5678,2786,20968,8502,2467,951,60647
1,8600000US60639,56047,2660,1982,2047,1374,1599,2526,14920,2286,2394,6906,2996,4930,1631,355,108,60639
1,8600000US60707,29881,522,645,400,628,419,907,7406,823,1531,5065,2493,5228,2054,481,158,60707
1,8600000US60622,40825,627,340,213,360,204,251,3352,492,950,3434,1493,17957,7025,2305,962,60622
1,8600000US60651,39140,935,1209,636,1259,1750,1378,11485,1255,2297,6233,2770,3721,1448,243,91,60651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,8600000US60623,50229,2134,1882,2407,1784,2296,1908,12209,1821,1801,6498,2405,3650,954,177,162,60623
1,8600000US60629,67343,2328,2099,1586,1864,1999,1853,23545,3060,2740,9152,3475,4702,1289,79,113,60629
1,8600000US60620,47028,328,799,652,1114,1735,1484,12691,1730,2964,11264,4067,4987,2217,290,162,60620
1,8600000US60637,27943,292,314,400,901,1027,579,5213,804,1660,5494,1630,3923,2897,803,1735,60637


In [43]:
# Consolidate and rename columns
selectB15003_full_df.rename(columns = {'B15003_001E':'Total_Pop', 'B15003_002E':'No_School','B15003_012E':'8th_Grade',
                                      'B15003_021E':'Assoc_Deg','B15003_022E':'Bach_Deg','B15003_023E':'Masters',
                                       'B15003_024E':'Prof_Deg','B15003_025E':'Doctorate'}, inplace = True)
selectB15003_full_df['Some_HS'] = selectB15003_full_df['B15003_013E'].astype('int64') + selectB15003_full_df['B15003_014E'].astype('int64')
+ selectB15003_full_df['B15003_015E'].astype('int64') + selectB15003_full_df['B15003_016E'].astype('int64')
selectB15003_full_df['HS_or_GED'] = selectB15003_full_df['B15003_017E'].astype('int64') + selectB15003_full_df['B15003_018E'].astype('int64')
selectB15003_full_df['Some_College'] = selectB15003_full_df['B15003_019E'].astype('int64') + selectB15003_full_df['B15003_020E'].astype('int64')

selectB15003_full_df

,GEO_ID,Total_Pop,No_School,8th_Grade,B15003_013E,B15003_014E,B15003_015E,B15003_016E,B15003_017E,B15003_018E,...,B15003_020E,Assoc_Deg,Bach_Deg,Masters,Prof_Deg,Doctorate,ZIP,Some_HS,HS_or_GED,Some_College
1,8600000US60647,62961,1020,1012,563,539,742,923,10657,1318,...,5678,2786,20968,8502,2467,951,60647,1102,11975,7744
1,8600000US60639,56047,2660,1982,2047,1374,1599,2526,14920,2286,...,6906,2996,4930,1631,355,108,60639,3421,17206,9300
1,8600000US60707,29881,522,645,400,628,419,907,7406,823,...,5065,2493,5228,2054,481,158,60707,1028,8229,6596
1,8600000US60622,40825,627,340,213,360,204,251,3352,492,...,3434,1493,17957,7025,2305,962,60622,573,3844,4384
1,8600000US60651,39140,935,1209,636,1259,1750,1378,11485,1255,...,6233,2770,3721,1448,243,91,60651,1895,12740,8530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,8600000US60623,50229,2134,1882,2407,1784,2296,1908,12209,1821,...,6498,2405,3650,954,177,162,60623,4191,14030,8299
1,8600000US60629,67343,2328,2099,1586,1864,1999,1853,23545,3060,...,9152,3475,4702,1289,79,113,60629,3450,26605,11892
1,8600000US60620,47028,328,799,652,1114,1735,1484,12691,1730,...,11264,4067,4987,2217,290,162,60620,1766,14421,14228
1,8600000US60637,27943,292,314,400,901,1027,579,5213,804,...,5494,1630,3923,2897,803,1735,60637,1301,6017,7154


In [ ]:
# Derive percentages

In [ ]:
# Trim columns

In [ ]:
# Export